# ロボットモデルの使い方

In [1]:
exec(open('/choreonoid_ws/install/share/irsl_choreonoid/sample/irsl_import.py').read())

## ロボットモデルのロード

In [2]:
fname = iu.parseURL('choreonoid://share/model/HRP4C/HRP4C.body')
if iu.isInChoreonoid():
    rb = ib.loadRobotItem(fname, world=False) ## in choreonoid GUI
else:
    rb = iu.loadRobot(fname) ## without GUI

## Link名とJoint名

In [3]:
if type(rb) is Body:
    for idx, lk in enumerate(rb.links):
        print('link {}: {}'.format(idx, lk.name))
    for idx, j in enumerate(rb.joints):
        print('joint {}: {}'.format(idx, j.jointName))
else:
    for idx, lk in enumerate(rb.body.links):
        print('link {}: {}'.format(idx, lk.name))
    for idx, j in enumerate(rb.body.joints):
        print('joint {}: {}'.format(idx, j.jointName))

link 0: WAIST
link 1: L_HIP_Y
link 2: L_HIP_R
link 3: L_HIP_P
link 4: L_KNEE_P
link 5: L_ANKLE_P
link 6: L_ANKLE_R
link 7: L_TOE_P
link 8: R_HIP_Y
link 9: R_HIP_R
link 10: R_HIP_P
link 11: R_KNEE_P
link 12: R_ANKLE_P
link 13: R_ANKLE_R
link 14: R_TOE_P
link 15: CHEST_P
link 16: CHEST_R
link 17: CHEST_Y
link 18: NECK_Y
link 19: NECK_R
link 20: NECK_P
link 21: L_SHOULDER_P
link 22: L_SHOULDER_R
link 23: L_SHOULDER_Y
link 24: L_ELBOW_P
link 25: L_WRIST_Y
link 26: L_WRIST_R
link 27: R_SHOULDER_P
link 28: R_SHOULDER_R
link 29: R_SHOULDER_Y
link 30: R_ELBOW_P
link 31: R_WRIST_Y
link 32: R_WRIST_R
joint 0: L_HIP_Y
joint 1: L_HIP_R
joint 2: L_HIP_P
joint 3: L_KNEE_P
joint 4: L_ANKLE_P
joint 5: L_ANKLE_R
joint 6: L_TOE_P
joint 7: R_HIP_Y
joint 8: R_HIP_R
joint 9: R_HIP_P
joint 10: R_KNEE_P
joint 11: R_ANKLE_P
joint 12: R_ANKLE_R
joint 13: R_TOE_P
joint 14: CHEST_P
joint 15: CHEST_R
joint 16: CHEST_Y
joint 17: NECK_Y
joint 18: NECK_R
joint 19: NECK_P
joint 20: L_SHOULDER_P
joint 21: L_SHOULDER_R

## IRSLのRobotModelクラスの使い方

### オブジェクトを作る

In [4]:
robot = RobotModel(rb)
robot.rootCoords(coordinates(robot.robot.rootLink.T))## HOT FIX これはいずれ必要なくなります

<coordinates[0x562d2b743410] 0 0 0.784 / 0 0 0 1 >

### jointのリスト

In [5]:
robot.jointList

[<cnoid.Body.Link named 'L_HIP_Y'>, <cnoid.Body.Link named 'L_HIP_R'>, <cnoid.Body.Link named 'L_HIP_P'>, <cnoid.Body.Link named 'L_KNEE_P'>, <cnoid.Body.Link named 'L_ANKLE_P'>, <cnoid.Body.Link named 'L_ANKLE_R'>, <cnoid.Body.Link named 'L_TOE_P'>, <cnoid.Body.Link named 'R_HIP_Y'>, <cnoid.Body.Link named 'R_HIP_R'>, <cnoid.Body.Link named 'R_HIP_P'>, <cnoid.Body.Link named 'R_KNEE_P'>, <cnoid.Body.Link named 'R_ANKLE_P'>, <cnoid.Body.Link named 'R_ANKLE_R'>, <cnoid.Body.Link named 'R_TOE_P'>, <cnoid.Body.Link named 'CHEST_P'>, <cnoid.Body.Link named 'CHEST_R'>, <cnoid.Body.Link named 'CHEST_Y'>, <cnoid.Body.Link named 'NECK_Y'>, <cnoid.Body.Link named 'NECK_R'>, <cnoid.Body.Link named 'NECK_P'>, <cnoid.Body.Link named 'L_SHOULDER_P'>, <cnoid.Body.Link named 'L_SHOULDER_R'>, <cnoid.Body.Link named 'L_SHOULDER_Y'>, <cnoid.Body.Link named 'L_ELBOW_P'>, <cnoid.Body.Link named 'L_WRIST_Y'>, <cnoid.Body.Link named 'L_WRIST_R'>, <cnoid.Body.Link named 'R_SHOULDER_P'>, <cnoid.Body.Link name

### angle-vector
全身の関節角度のベクトル

In [6]:
vec = robot.angleVector()
vec

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

angle-vectorをセットする

In [7]:
vec[2] = -0.3
robot.angleVector(vec)

### 関節への名前アクセス

In [8]:
robot.jointAngle('L_HIP_P')

-0.3

In [9]:
robot.jointAngle('L_HIP_P', 0.0)

0.0

### 関節への辞書型でのアクセス

In [10]:
robot.setAngleMap({'R_HIP_P': -0.8, 'R_KNEE_P': 1.6})

True

### ロボットの座標系
ロボットの座標系は上部をZ軸とし、前方をX軸とする。

ワールド座標系は、Z軸方向が鉛直上向きとする。

In [11]:
robot.rootCoords()

<coordinates[0x562d2b743410] 0 0 0.784 / 0 0 0 1 >

In [12]:
cds = coordinates(npa([0, 0, 1.0]))
robot.rootCoords(cds)

<coordinates[0x562d2b76b390] 0 0 1 / 0 0 0 1 >

### 四肢(limb), 効果器(EndEffector), 関節グループ
関節のまとまりで、先端にエンドエフェクターを持つものを定義する。

これを便宜上、四肢(limb)と名づける。

名前は自由だが、arm, rarm, larm, rleg, lleg, neck, torso はアクセサが準備されている。

In [13]:
robot.registerEndEffector('larm', ## end-effector
                          'L_WRIST_R', ## tip-link
                          joint_tuples = (('L_SHOULDER_P', 'shoulder-p'),
                                          ('L_SHOULDER_R', 'shoulder-r'),
                                          ('L_SHOULDER_Y', 'shoulder-y'),
                                          ('L_ELBOW_P', 'elbow-p'),
                                          ('L_WRIST_Y', 'wrist-y'),
                                          ('L_WRIST_R', 'wrist-r'),
                                          )
                          )
robot.registerEndEffector('rarm', ## end-effector
                          'R_WRIST_R', ## tip-link
                          joint_tuples = (('R_SHOULDER_P', 'shoulder-p'),
                                          ('R_SHOULDER_R', 'shoulder-r'),
                                          ('R_SHOULDER_Y', 'shoulder-y'),
                                          ('R_ELBOW_P', 'elbow-p'),
                                          ('R_WRIST_Y', 'wrist-y'),
                                          ('R_WRIST_R', 'wrist-r'),
                                          )
                          )

In [14]:
robot.larm

#### limbのangle-vector

In [15]:
lvec = robot.larm.angleVector()
lvec

array([0., 0., 0., 0., 0., 0.])

In [17]:
lvec[0] = -0.4
robot.larm.angleVector(lvec)

array([-0.4,  0. ,  0. ,  0. ,  0. ,  0. ])

#### limbの名前アクセス
ニックネームを付けることができる

In [18]:
robot.larm.jointAngle('shoulder-p')

-0.4

In [19]:
robot.larm.setAngleMap({'shoulder-p': 0.4, 'elbow-p': -0.8})

True

In [21]:
robot.rarm.setAngleMap({'shoulder-p': 0.4, 'elbow-p': -0.8})

True

#### 手先効果器(EndEffector)

In [22]:
robot.larm.endEffector

<coordinates[0x562d2ca2efe0] -0.0548972 0.198627 1.01957 / 0.0854184 -0.197913 0.0173152 0.976337 >

In [25]:
dc = DrawCoords(width=5)

In [26]:
dc.addCoords(robot.larm.endEffector)

1

#### 逆運動学

In [27]:
tgt = robot.larm.endEffector

In [28]:
robot.larm.inverseKinematics(tgt, constraint='xyz')

(True, 0)

ターゲットを動かしてみる

In [29]:
tgt.translate(npa([0, 0, 0.2]), coordinates.wrt.world)

<coordinates[0x562d285f11d0] -0.0548972 0.198627 1.21957 / 0.0854184 -0.197913 0.0173152 0.976337 >

In [30]:
robot.larm.inverseKinematics(tgt, constraint='xyz')

(True, 4)

In [31]:
dc.reset()
dc.addCoords(robot.larm.endEffector)

1

### 名前付きのポーズの設定

In [32]:
robot.registerNamedPose('default',
                        [0, 0, 0, 0, 0, 0, 0, ## LLEG
                         0, 0, 0, 0, 0, 0, 0, ## RLEG
                         0, 0, 0, ## CHEST
                         0, 0, 0, ## NECK
                         0, 0, 0, 0, 0, 0, ## LARM
                         0, 0, 0, 0, 0, 0, ## RARM
                         ])

In [33]:
robot.setDefaultPose()

## ロボットの独自クラスを作る

In [37]:
class HRP4CModel(RobotModel):
    def __init__(self):
        fname = iu.parseURL('choreonoid://share/model/HRP4C/HRP4C.body')
        if iu.isInChoreonoid():
            rb = ib.loadRobotItem(fname, world=False)
        else:
            rb = iu.loadRobot(fname)
        super().__init__(rb)
        self.rootCoords(coordinates(self.robot.rootLink.T))## HOT FIX これはいずれ必要なくなります
        self.registerEndEffector('larm', ## end-effector
                                 'L_WRIST_R', ## tip-link
                                 joint_tuples = (('L_SHOULDER_P', 'shoulder-p'),
                                                 ('L_SHOULDER_R', 'shoulder-r'),
                                                 ('L_SHOULDER_Y', 'shoulder-y'),
                                                 ('L_ELBOW_P', 'elbow-p'),
                                                 ('L_WRIST_Y', 'wrist-y'),
                                                 ('L_WRIST_R', 'wrist-r'),
                                                 )
                                 )
        self.registerEndEffector('rarm', ## end-effector
                                 'R_WRIST_R', ## tip-link
                                 joint_tuples = (('R_SHOULDER_P', 'shoulder-p'),
                                                 ('R_SHOULDER_R', 'shoulder-r'),
                                                 ('R_SHOULDER_Y', 'shoulder-y'),
                                                 ('R_ELBOW_P', 'elbow-p'),
                                                 ('R_WRIST_Y', 'wrist-y'),
                                                 ('R_WRIST_R', 'wrist-r'),
                                                 )
                                 )
        self.registerNamedPose('default',
                               [0, 0, 0, 0, 0, 0, 0, ## LLEG
                                0, 0, 0, 0, 0, 0, 0, ## RLEG
                                0, 0, 0, ## CHEST
                                0, 0, 0, ## NECK
                                0, 0, 0, 0, 0, 0, ## LARM
                                0, 0, 0, 0, 0, 0, ## RARM
                                ])
        self.setDefaultPose()

まえに表示したモデルは消しておく

In [38]:
robot = HRP4CModel()